In [14]:
import os
import json

from itertools import groupby
from config import iam_policy_data
import preprocessiong_iam_policy

def group_permissions_by_service(policy_lists):
    grouped_permissions = {}
    for policy_list in policy_lists:
        if policy_list["Service Effect"] != iam_policy_data.service_effect[0]:
            service = policy_list["Service"]
            if service not in grouped_permissions:
                grouped_permissions[service] = {"Action": [], "Resource": "*"}
            grouped_permissions[service]["Action"].append(policy_list["Actions"])
        else:
            pass
    return grouped_permissions

def generate_iam_policy(grouped_permissions):
    iam_policy = {
        "Version": "2012-10-17",
        "Statement": [],
    }

    for service, actions_resources in grouped_permissions.items():
        statement = {
            "Sid": f"{service.lower()}allowfordevops",
            "Effect": "Allow",
            "Action": actions_resources["Action"],
            "Resource": actions_resources["Resource"],
        }
        iam_policy["Statement"].append(statement)

    return iam_policy

def save_iam_policy_to_file(iam_policy, category, output_directory):
    output_file_path = os.path.join(output_directory, f"{category.lower()}_iam_policy_devops")
    with open(output_file_path, 'w') as json_file:
        json.dump(iam_policy, json_file, indent=4)

# def main():
#     policy_lists = preprocessiong_iam_policy.excel_to_json(iam_policy_data.input_file_path, iam_policy_data.sheet_name)
#     grouped_by_category = {key: list(group) for key, group in groupby(policy_lists, key=lambda x: x['Category'])}
    
#     for category, items in grouped_by_category.items():
#         print(f"Category: {category}")
#         grouped_permissions = group_permissions_by_service(items)
#         iam_policy = generate_iam_policy(grouped_permissions)    
#         save_iam_policy_to_file(iam_policy, category , iam_policy_data.output_directory)

# if __name__ == "__main__":
#     main()


In [15]:
policy_lists = preprocessiong_iam_policy.excel_to_json(iam_policy_data.input_file_path, iam_policy_data.sheet_name)

In [16]:
# Category와 Wildcard로 데이터 정렬
sorted_data = sorted(policy_lists, key=lambda x: (x['Category'], x['Wildcard']))

# Category와 Wildcard로 그룹화
grouped_by_category_and_wildcard = {key: list(group) for key, group in groupby(sorted_data, key=lambda x: (x['Category'], x['Wildcard']))}

# Grouped 결과를 저장할 리스트
grouped_actions = []
personal_actions = []

# 그룹화된 데이터에서 Service Effect가 모두 같은 경우 Action을 하나로 처리
for key, group in grouped_by_category_and_wildcard.items():
    category, wildcard = key
    service_effects = set(item['Service Effect'] for item in group)
    if iam_policy_data.service_effect[0] not in service_effects:
        action = f"{group[0]['Actions'].split(':')[0].lower()}:{wildcard}"
        grouped_actions.append({'Wildcard' : wildcard, 'Action': action, 'Category': category, 'Service': group[0]['Service'], 'Service Effect': service_effects.pop()})

# # Action 값의 ":" 뒤의 값이 policy_lists의 Wildcard 값과 다른 경우 추가
# for item in policy_lists:
#     action_name = item['Actions'].split(':')[1]
#     if any(action_name not in grouped_item['Action'] for grouped_item in grouped_actions):
#         grouped_actions.append(item)


In [17]:
grouped_by_category_and_wildcard

{('Infra',
  'Accept*'): [{'Wildcard': 'Accept*',
   'Actions': 'ec2:AcceptAddressTransfer',
   'Category': 'Infra',
   'Service': 'EC2',
   'Service Effect': '상 (높음)'}, {'Wildcard': 'Accept*',
   'Actions': 'ec2:AcceptReservedInstancesExchangeQuote',
   'Category': 'Infra',
   'Service': 'EC2',
   'Service Effect': '중 (중간)'}, {'Wildcard': 'Accept*',
   'Actions': 'ec2:AcceptVpcPeeringConnection',
   'Category': 'Infra',
   'Service': 'VPC',
   'Service Effect': '중 (중간)'}],
 ('Infra',
  'Allocate*'): [{'Wildcard': 'Allocate*',
   'Actions': 'ec2:AllocateAddress',
   'Category': 'Infra',
   'Service': 'EC2',
   'Service Effect': '중 (중간)'}, {'Wildcard': 'Allocate*',
   'Actions': 'ec2:AllocateHosts',
   'Category': 'Infra',
   'Service': 'EC2',
   'Service Effect': '중 (중간)'}],
 ('Infra',
  'Assign*'): [{'Wildcard': 'Assign*',
   'Actions': 'ec2:AssignIpv6Addresses',
   'Category': 'Infra',
   'Service': 'VPC',
   'Service Effect': '중 (중간)'}, {'Wildcard': 'Assign*',
   'Actions': 'ec2:Ass

In [18]:
#### grouped_actions_dict를 'Wildcard'를 키로 하는 딕셔너리로 변경
grouped_actions_dict_by_wildcard = {item['Wildcard']: item for item in grouped_actions}

# 삭제할 인덱스를 저장할 리스트 초기화
indices_to_remove = []

# policy_lists 순회
for idx, policy in enumerate(policy_lists):
    wildcard = policy.get('Wildcard')
    category = policy.get('Category')
    service = policy.get('Service')
    print(category)

    # wildcard이 grouped_actions_dict에 존재하면서 값이 일치하는 경우
    try:
        if grouped_actions_dict_by_wildcard[wildcard]['Wildcard'] == wildcard and grouped_actions_dict_by_wildcard[wildcard]['Category'] == category and grouped_actions_dict_by_wildcard[wildcard]['Service'] == service:
            indices_to_remove.append(idx)
            print(idx)
    except KeyError:
        # KeyError가 발생하면 예외 처리
        print(f"KeyError: {wildcard} not found in grouped_actions_dict_by_wildcard")

# 저장된 인덱스를 역순으로 정렬하여 뒤에서부터 삭제
for idx in reversed(indices_to_remove):
    del policy_lists[idx]


Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Infra
KeyError: Accept* not found in grouped_actions_dict_by_wildcard
Network
KeyError: Activate* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Security
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add* not found in grouped_actions_dict_by_wildcard
Storage
KeyError: Add

In [19]:
grouped_actions_dict_by_wildcard = {item['Wildcard']: item for item in grouped_actions}

In [20]:
# grouped_actions_dict_by_wildcard
output_file_path = os.path.join('./', "group_test.json")
with open(output_file_path, 'w') as json_file:
    json.dump(grouped_actions_dict_by_wildcard, json_file, indent=4)

In [21]:
policy_lists[200:250]

[{'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteRoute',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteRouteTable',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteSubnet',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteSubnetCidrReservation',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteTrafficMirrorFilter',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteTrafficMirrorFilterRule',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 'Delete*',
  'Actions': 'ec2:DeleteTrafficMirrorSession',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '상 (높음)'},
 {'Wildcard': 

In [22]:
grouped_actions

[{'Wildcard': 'Allocate*',
  'Action': 'ec2:Allocate*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Assign*',
  'Action': 'ec2:Assign*',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Attach*',
  'Action': 'ec2:Attach*',
  'Category': 'Infra',
  'Service': 'VPC',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Confirm*',
  'Action': 'ec2:Confirm*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Copy*',
  'Action': 'ec2:Copy*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Describe*',
  'Action': 'ec2:Describe*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '하 (낮음)'},
 {'Wildcard': 'Disable*',
  'Action': 'ec2:Disable*',
  'Category': 'Infra',
  'Service': 'EC2',
  'Service Effect': '중 (중간)'},
 {'Wildcard': 'Discover*',
  'Action': 'ecs:Discover*',
  'Category': 'Infra',
  'Service': 'ECS',
  'Service 

In [23]:
print(len(policy_lists))
print(len(grouped_actions))

953
39


In [24]:
441 + 67

508

In [25]:
policy_lists.extend(grouped_actions)

In [26]:
len(policy_lists)

992

In [27]:
group_dict = preprocessiong_iam_policy.group_dict_by_key(policy_lists, 'Category')

In [28]:
for category, items in group_dict.items():
    print(f"{category} IAM Policy created for devops")
    grouped_permissions = group_permissions_by_service(items)
    iam_policy = generate_iam_policy(grouped_permissions)    
    save_iam_policy_to_file(iam_policy, category, iam_policy_data.output_directory)


Infra IAM Policy created for devops


KeyError: 'Actions'

In [2]:
## 구글 시트 연동

In [5]:
#!pip3 install --upgrade oauth2client 
#!pip3 install gspread
#!pip3 install PyOpenSSL


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [18]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = 'https://spreadsheets.google.com/feeds'
json = './python-iam-policy-update-d8f00ffa6f0b.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json, scope)
gc = gspread.authorize(credentials)
sheet_url = 'https://docs.google.com/spreadsheets/d/1N6EiipYeeQdEKR24mvaAFm3jAJF3d8yDNPVc47ZqgRU/edit?usp=sharing'
doc = gc.open_by_url(sheet_url)
worksheet = doc.worksheet('AWS_api_operations')

# 데이터프레임으로 변환
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])
json_file = df[["Wildcard", "Actions", "Category", "Service", "Service Effect"]].to_dict(orient="records")
# 데이터프레임 출력
# cell_data = worksheet.acell('값을가져올 셀이름').value
# print(cell_data)

In [26]:
import json

def find_json_differences(file1_path, file2_path):
    # JSON 파일을 읽어옵니다.
    with open(file1_path, 'r', encoding='utf-8') as file1, open(file2_path, 'r', encoding='utf-8') as file2:
        json1 = json.load(file1)
        json2 = json.load(file2)
    
    # 두 JSON 파일을 비교하여 차이점을 찾습니다.
    differences = {}
    for key, value in json1.items():
        if key == 'Statement':
            for i in range(len(value)):
                for action in value[i]['Action']:
                    if action not in json2[key][i]['Action']:
                        if key not in differences:
                            differences[key] = {}
                        if i not in differences[key]:
                            differences[key][i] = []
                        differences[key][i].append(action)

    # 차이점을 출력합니다.
    if differences:
        print("두 파일의 차이점:")
        for key, value in differences.items():
            for i, actions in value.items():
                for action in actions:
                    print(f"  {key}[{i}]의 Action에 차이가 있습니다: {action}")
    else:
        print("두 파일은 동일합니다.")

file1_path = './iam_policy_devops_infra.json'
file2_path = './terraform/#policy/iam_policy_devops_infra.json'

# JSON 파일 비교 함수를 호출합니다.
find_json_differences(file1_path, file2_path)


두 파일의 차이점:
  Statement[0]의 Action에 차이가 있습니다: ec2:CreateTags
  Statement[2]의 Action에 차이가 있습니다: eks:CreateAddon
  Statement[2]의 Action에 차이가 있습니다: eks:CreateCluster
  Statement[2]의 Action에 차이가 있습니다: eks:CreateFargateProfile
  Statement[2]의 Action에 차이가 있습니다: eks:CreateNodegroup
  Statement[2]의 Action에 차이가 있습니다: eks:Describe*
  Statement[2]의 Action에 차이가 있습니다: eks:List*
  Statement[2]의 Action에 차이가 있습니다: eks:Tag*
  Statement[3]의 Action에 차이가 있습니다: lambda:Add*
  Statement[3]의 Action에 차이가 있습니다: lambda:Create*
  Statement[3]의 Action에 차이가 있습니다: lambda:Get*
  Statement[3]의 Action에 차이가 있습니다: lambda:Invoke*
  Statement[3]의 Action에 차이가 있습니다: lambda:List*
  Statement[3]의 Action에 차이가 있습니다: lambda:Publish*
  Statement[3]의 Action에 차이가 있습니다: lambda:Put*
  Statement[3]의 Action에 차이가 있습니다: lambda:Tag*
  Statement[3]의 Action에 차이가 있습니다: lambda:Untag*
  Statement[3]의 Action에 차이가 있습니다: lambda:Update*


In [25]:
import json

def find_json_differences(original_path, modified_path):
    # JSON 파일을 읽어옵니다.
    with open(original_path, 'r', encoding='utf-8') as original_file, open(modified_path, 'r', encoding='utf-8') as modified_file:
        original_json = json.load(original_file)
        modified_json = json.load(modified_file)
    
    # 두 JSON 파일을 비교하여 차이점을 찾습니다.
    differences = {}
    for key, value in modified_json.items():
        if key == 'Statement':
            for i in range(len(value)):
                for action in value[i]['Action']:
                    if action not in original_json[key][i]['Action']:
                        if key not in differences:
                            differences[key] = {}
                        if i not in differences[key]:
                            differences[key][i] = []
                        differences[key][i].append(action)

    # 차이점을 출력합니다.
    if differences:
        print("원본 파일과 수정 파일의 차이점:")
        for key, value in differences.items():
            for i, actions in value.items():
                for action in actions:
                    print(f"  {key}[{i}]의 Action에 차이가 있습니다: {action}")
    else:
        print("원본 파일과 수정 파일은 동일합니다.")

# 비교할 두 JSON 파일 경로를 지정합니다.
original_path = './iam_policy_devops_infra.json'
modified_path = './terraform/#policy/iam_policy_devops_infra.json'

# JSON 파일 비교 함수를 호출합니다.
find_json_differences(original_path, modified_path)


IndexError: list index out of range

In [28]:
original_path = './iam_policy_devops_infra.json'
modified_path = './terraform/#policy/iam_policy_devops_infra.json'

with open(original_path, 'r', encoding='utf-8') as original_file, open(modified_path, 'r', encoding='utf-8') as modified_file:
    original_json = json.load(original_file)
    modified_json = json.load(modified_file)

In [47]:
## 여기서부터 업데이트 코드작성 중

In [2]:
import json

def compare_sid_and_actions(original_path, modified_path):
    # JSON 파일을 읽어옵니다.
    with open(original_path, 'r', encoding='utf-8') as original_file, open(modified_path, 'r', encoding='utf-8') as modified_file:
        original_json = json.load(original_file)
        modified_json = json.load(modified_file)
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 추출
    original_statements = original_json.get("Statement", [])
    modified_statements = modified_json.get("Statement", [])
    
    original_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in original_statements}
    modified_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in modified_statements}
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 차이 비교
    added_sids = [sid for sid in modified_sid_actions.keys() if sid not in original_sid_actions]
    removed_sids = [sid for sid in original_sid_actions.keys() if sid not in modified_sid_actions]
    
    # 결과 출력
    print("추가된 Sid와 해당 Action 값:")
    if added_sids:
        for sid in added_sids:
            print(f"  Sid: {sid}")
            print(f"    Action: {json.dumps(modified_sid_actions[sid], indent=4)}")
    else:
        print("없음")
    
    print("\n빠진 Sid와 해당 Action 값:")
    if removed_sids:
        for sid in removed_sids:
            print(f"  Sid: {sid}")
            print(f"    Action: {json.dumps(original_sid_actions[sid], indent=4)}")
    else:
        print("없음")

# 비교할 두 JSON 파일 경로를 지정합니다.
original_path = './iam_policy_devops_infra.json'
modified_path = './terraform/#policy/iam_policy_devops_infra.json'

# JSON 파일의 추가된 Sid와 빠진 Sid와 해당 Action 값 비교 함수를 호출합니다.
compare_sid_and_actions(original_path, modified_path)


추가된 Sid와 해당 Action 값:
  Sid: LambdaForDevOps
    Action: [
    "lambda:Get*",
    "lambda:List*",
    "lambda:CreateAlias",
    "lambda:DeleteFunction"
]
  Sid: RDSForDevOps
    Action: [
    "rds:*"
]
  Sid: TagForDevOps
    Action: [
    "tag:getResources",
    "tag:getTagKeys",
    "tag:getTagValues",
    "tag:TagResources",
    "tag:UntagResources"
]
  Sid: ComputerOptimizer
    Action: [
    "compute-optimizer:*"
]
  Sid: SNSandChatbotforDevOps
    Action: [
    "SNS:*",
    "chatbot:*"
]

빠진 Sid와 해당 Action 값:
  Sid: eksallowfordevops
    Action: [
    "eks:CreateAddon",
    "eks:CreateCluster",
    "eks:CreateFargateProfile",
    "eks:CreateNodegroup",
    "eks:Describe*",
    "eks:List*",
    "eks:Tag*"
]
  Sid: lambdaallowfordevops
    Action: [
    "lambda:DeleteFunction",
    "lambda:Add*",
    "lambda:Create*",
    "lambda:Get*",
    "lambda:Invoke*",
    "lambda:List*",
    "lambda:Publish*",
    "lambda:Put*",
    "lambda:Tag*",
    "lambda:Untag*",
    "lambda:Update*"
]


In [48]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = 'https://spreadsheets.google.com/feeds'
json = './python-iam-policy-update-d8f00ffa6f0b.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json, scope)
gc = gspread.authorize(credentials)
sheet_url = 'https://docs.google.com/spreadsheets/d/1N6EiipYeeQdEKR24mvaAFm3jAJF3d8yDNPVc47ZqgRU/edit?usp=sharing'
doc = gc.open_by_url(sheet_url)
worksheet = doc.worksheet('AWS_api_operations')

In [1]:
import re
import json
import time

def compare_sid_and_actions(original_path, modified_path):
    # JSON 파일을 읽어옵니다.
    with open(original_path, 'r', encoding='utf-8') as original_file, open(modified_path, 'r', encoding='utf-8') as modified_file:
        original_json = json.load(original_file)
        modified_json = json.load(modified_file)
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 추출
    original_statements = original_json.get("Statement", [])
    modified_statements = modified_json.get("Statement", [])
    
    original_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in original_statements}
    modified_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in modified_statements}
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 차이 비교
    added_sids = [sid for sid in modified_sid_actions.keys() if sid not in original_sid_actions]
    removed_sids = [sid for sid in original_sid_actions.keys() if sid not in modified_sid_actions]
    
    # 추가된 Sid와 해당 Action 값 반환
    added_sid_actions = {}
    for sid in added_sids:
        added_sid_actions[sid] = modified_sid_actions[sid]
    
    return added_sid_actions

def convert_pattern(pattern_string):
    # '*'를 '.'으로 대체하고 '^'와 '$'를 문자열의 시작과 끝으로 추가하여 변환
    converted_pattern = re.sub(r'\*', '.*', pattern_string)
    converted_pattern = f'^{converted_pattern}$'
    return converted_pattern

# 비교할 두 JSON 파일 경로를 지정합니다.
original_path = './iam_policy_devops_infra.json'
modified_path = './terraform/#policy/iam_policy_devops_infra.json'

# JSON 파일의 추가된 Sid와 해당 Action 값 비교 함수를 호출합니다.
# compare_origin_and_modify(original_path, modified_path)
added_sid_actions = compare_sid_and_actions(original_path, modified_path)

# 구글 시트에 연결
scope = scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_keyfile = './python-iam-policy-update-d8f00ffa6f0b.json'  # JSON 키 파일의 경로
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
gc = gspread.authorize(credentials)

# 구글 시트 문서 및 워크시트 열기
sheet_url = 'https://docs.google.com/spreadsheets/d/1N6EiipYeeQdEKR24mvaAFm3jAJF3d8yDNPVc47ZqgRU/edit?usp=sharing'
doc = gc.open_by_url(sheet_url)
worksheet = doc.worksheet('AWS_api_operations')

update_count = 0

# 추가된 Sid와 해당 Action 값이 포함된지 확인하고 업데이트
for sid, action_list in added_sid_actions.items():    
    for action in action_list:
        try:
            cell = worksheet.find(action)
            worksheet.update_cell(cell.row, cell.col + 4, action)  # Action 값을 Sid 옆의 다음 열에 업데이트
            print(f"{action} Update Complete")
            update_count += 1

        except Exception as e:
            print(f"The {action} is wildcard type. Bring the {action} policy all list.")
            
            converted_pattern = convert_pattern(action)
            wildcard_policy_all = re.compile(converted_pattern)

            cell_list = worksheet.findall(wildcard_policy_all)

            if cell_list != []:
                for cell in cell_list:
                    worksheet.find(cell.value)
                    if worksheet.row_values(cell.row)[6] == '상':
                        worksheet.update_cell(cell.row, cell.col + 4, '중')
                    print(f"{cell.value} Update Complete")
                    update_count += 1
                    if update_count >= 20:
                        print("20 updates completed. Waiting for 1 minute...")
                        for i in range(60):
                            print(f"{i+1} seconds passed.")
                            time.sleep(1)
                        print("1 minute has passed.")
                        update_count = 0    # 업데이트 카운트 초기화
            else:
                new_row_data = ['', '', action, '', '', '', '중', 'test']
                worksheet.append_row(new_row_data)


The lambda:Get* is wildcard type. Bring the lambda:Get* policy all list.
lambda:GetAccountSettings Update Complete
lambda:GetAlias Update Complete
lambda:GetCodeSigningConfig Update Complete
lambda:GetEventSourceMapping Update Complete
lambda:GetFunction Update Complete
lambda:GetFunctionCodeSigningConfig Update Complete
lambda:GetFunctionConcurrency Update Complete
lambda:GetFunctionConfiguration Update Complete
lambda:GetFunctionEventInvokeConfig Update Complete
lambda:GetFunctionUrlConfig Update Complete
lambda:GetLayerVersion Update Complete
lambda:GetLayerVersionByArn Update Complete
lambda:GetLayerVersionPolicy Update Complete
lambda:GetPolicy Update Complete
lambda:GetProvisionedConcurrencyConfig Update Complete
lambda:GetRuntimeManagementConfig Update Complete
The lambda:List* is wildcard type. Bring the lambda:List* policy all list.
lambda:ListAliases Update Complete
lambda:ListCodeSigningConfigs Update Complete
lambda:ListEventSourceMappings Update Complete
lambda:ListFunctio

KeyboardInterrupt: 

In [ ]:
# 추가된 Sid와 해당 Action 값이 포함된지 확인하고 업데이트
for sid, action_list in added_sid_actions.items():
    # cell = worksheet.find(action_list)
    for action in action_list:
        # print(action)
        try:
            cell = worksheet.find(action)
            worksheet.update_cell(cell.row, cell.col + 4, action)  # Action 값을 Sid 옆의 다음 열에 업데이트
            print(f"{action} Update Complete")
        except Exception as e:
            print(f"The {action} is wildcard type. Bring the {action} policy all list.")
            
            converted_pattern = convert_pattern(action)
            wildcard_policy_all = re.compile(converted_pattern)

            cell_list = worksheet.findall(wildcard_policy_all)

            if cell_list != []:
                for cell in cell_list:
                    worksheet.find(cell.value)
                    if worksheet.row_values(cell.row)[6] == '상':
                        worksheet.update_cell(cell.row, cell.col + 4, '중')
                    print(f"{cell.value} Update Complete")
            else:
                new_row_data = ['', '', action, '', '', '', '중', 'test']
                worksheet.append_row(new_row_data)

In [11]:
worksheet = doc.worksheet('AWS_api_operations')

In [176]:
cell_list[0].row

155

In [181]:
worksheet.row_values(90)

['Describe*',
 'DescribeOrderableDBInstanceOptions',
 'rds:DescribeOrderableDBInstanceOptions',
 'Storage',
 'RDS',
 'RDS OptionGroup',
 '하',
 '지정된 DB 엔진, 엔진 버전 및 인스턴스 클래스에 대해 주문 가능한 옵션 목록 반환']

In [6]:
import re

def convert_pattern(pattern_string):
    # '*'를 '.'으로 대체하고 '^'와 '$'를 문자열의 시작과 끝으로 추가하여 변환
    converted_pattern = re.sub(r'\*', '.*', pattern_string)
    converted_pattern = f'^{converted_pattern}$'
    return converted_pattern

# 변환할 패턴
original_pattern = 'rds:*'

# 변환된 패턴
converted_pattern = convert_pattern(original_pattern)

converted_pattern

'^rds:.*$'

In [12]:
# 정규 표현식 패턴 설정
pattern = re.compile(converted_pattern)

# 특정 패턴을 포함하는 모든 셀을 찾기
cell_list = worksheet.findall(pattern)


<Worksheet 'Lambda_api_operations' id:1737499664>

In [14]:
for i,cell in enumerate(cell_list):
    print(i, cell.value)
    

0 rds:AddRoleToDBCluster
1 rds:AddRoleToDBInstance
2 rds:AddSourceIdentifierToSubscription
3 rds:AddTagsToResource
4 rds:ApplyPendingMaintenanceAction
5 rds:AuthorizeDBSecurityGroupIngress
6 rds:BacktrackDBCluster
7 rds:CancelExportTask
8 rds:CopyDBClusterParameterGroup
9 rds:CopyDBClusterSnapshot
10 rds:CopyDBParameterGroup
11 rds:CopyDBSnapshot
12 rds:CopyOptionGroup
13 rds:CreateBlueGreenDeployment
14 rds:CreateCustomDBEngineVersion
15 rds:CreateDBCluster
16 rds:CreateDBClusterEndpoint
17 rds:CreateDBClusterParameterGroup
18 rds:CreateDBClusterSnapshot
19 rds:CreateDBInstance
20 rds:CreateDBInstanceReadReplica
21 rds:CreateDBParameterGroup
22 rds:CreateDBProxy
23 rds:CreateDBProxyEndpoint
24 rds:CreateDBSecurityGroup
25 rds:CreateDBSnapshot
26 rds:CreateDBSubnetGroup
27 rds:CreateEventSubscription
28 rds:CreateGlobalCluster
29 rds:CreateIntegration
30 rds:CreateOptionGroup
31 rds:CreateTenantDatabase
32 rds:DeleteBlueGreenDeployment
33 rds:DeleteCustomDBEngineVersion
34 rds:DeleteDB

In [102]:
cell = worksheet.find("lambda:DeleteAlias")

In [103]:
cell

<Cell R413C3 'lambda:DeleteAlias'>

In [104]:
worksheet.update_cell(cell.row, cell.col + 4, "중")

{'spreadsheetId': '1N6EiipYeeQdEKR24mvaAFm3jAJF3d8yDNPVc47ZqgRU',
 'updatedRange': 'AWS_api_operations!G413',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [101]:
action

'chatbot:*'

In [12]:
import os

file_list = os.listdir("./")
original_files = [file for file in file_list if file.endswith('.json')]

original_files

['iam_policy_devops_resource.json',
 'iam_policy_devops_network.json',
 'iam_policy_devops_mornitoring.json',
 'iam_policy_devops_account.json',
 'iam_policy_devops_storage.json',
 'iam_policy_devops_security.json',
 'iam_policy_devops_infra.json']

In [10]:
import re
import json
import time

def compare_sid_and_actions(original_path, modified_path):
    # JSON 파일을 읽어옵니다.
    with open(original_path, 'r', encoding='utf-8') as original_file, open(modified_path, 'r', encoding='utf-8') as modified_file:
        original_json = json.load(original_file)
        modified_json = json.load(modified_file)
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 추출
    original_statements = original_json.get("Statement", [])
    modified_statements = modified_json.get("Statement", [])
    
    original_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in original_statements}
    modified_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in modified_statements}
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 차이 비교
    added_sids = [sid for sid in modified_sid_actions.keys() if sid not in original_sid_actions]
    removed_sids = [sid for sid in original_sid_actions.keys() if sid not in modified_sid_actions]
    
    # 추가된 Sid와 해당 Action 값 반환
    added_sid_actions = {}
    for sid in added_sids:
        added_sid_actions[sid] = modified_sid_actions[sid]
    
    return added_sid_actions

def convert_pattern(pattern_string):
    # '*'를 '.'으로 대체하고 '^'와 '$'를 문자열의 시작과 끝으로 추가하여 변환
    converted_pattern = re.sub(r'\*', '.*', pattern_string)
    converted_pattern = f'^{converted_pattern}$'
    return converted_pattern

# 비교할 두 JSON 파일 경로를 지정합니다.
original_path = './iam_policy_devops_infra.json'
modified_path = './terraform/#policy/iam_policy_devops_infra.json'

# JSON 파일의 추가된 Sid와 해당 Action 값 비교 함수를 호출합니다.
# compare_origin_and_modify(original_path, modified_path)
added_sid_actions = compare_sid_and_actions(original_path, modified_path)

# 구글 시트에 연결
scope = scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_keyfile = './python-iam-policy-update-d8f00ffa6f0b.json'  # JSON 키 파일의 경로
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
gc = gspread.authorize(credentials)

# 구글 시트 문서 및 워크시트 열기
sheet_url = 'https://docs.google.com/spreadsheets/d/1N6EiipYeeQdEKR24mvaAFm3jAJF3d8yDNPVc47ZqgRU/edit?usp=sharing'
doc = gc.open_by_url(sheet_url)
worksheet = doc.worksheet('AWS_api_operations')

update_count = 0

# 추가된 Sid와 해당 Action 값이 포함된지 확인하고 업데이트
for sid, action_list in added_sid_actions.items():    
    for action in action_list:
        try:
            cell = worksheet.find(action)
            worksheet.update_cell(cell.row, cell.col + 4, action)  # Action 값을 Sid 옆의 다음 열에 업데이트
            print(f"{action} Update Complete")
            update_count += 1

        except Exception as e:
            print(f"The {action} is wildcard type. Bring the {action} policy all list.")
            
            converted_pattern = convert_pattern(action)
            wildcard_policy_all = re.compile(converted_pattern)

            cell_list = worksheet.findall(wildcard_policy_all)

            if cell_list != []:
                for cell in cell_list:
                    worksheet.find(cell.value)
                    if worksheet.row_values(cell.row)[6] == '상':
                        worksheet.update_cell(cell.row, cell.col + 4, '중')
                    print(f"{cell.value} Update Complete")
                    update_count += 1
                    if update_count >= 20:
                        print("20 updates completed. Waiting for 1 minute...")
                        for i in range(60):
                            print(f"{i+1} seconds passed.")
                            time.sleep(1)
                        print("1 minute has passed.")
                        update_count = 0    # 업데이트 카운트 초기화
            else:
                new_row_data = ['', '', action, '', '', '', '중', 'test']
                worksheet.append_row(new_row_data)


NameError: name 'ServiceAccountCredentials' is not defined

In [1]:
import preprocessiong_iam_policy as pre_iam
import config

In [2]:
original_files=pre_iam.policy_file_name_list(config.iam_policy_data.original_path)
modify_files=pre_iam.policy_file_name_list(config.iam_policy_data.modified_path)

In [3]:
original_files

['iam_policy_devops_resource.json',
 'iam_policy_devops_network.json',
 'iam_policy_devops_mornitoring.json',
 'iam_policy_devops_account.json',
 'iam_policy_devops_storage.json',
 'iam_policy_devops_security.json',
 'iam_policy_devops_infra.json']

In [4]:
modify_files

['adena_devops_biling_policy.json',
 'iam_policy_devops_resource.json',
 'adena_devops_policy.json',
 'iam_policy_devops_mornitoring(instaed of CloudWatchFullAccessV2).json',
 'iam_policy_devops_network.json',
 'iam_policy_devops_account.json',
 'iam_policy_devops_storage.json',
 'iam_policy_devops_security.json',
 'iam_policy_devops_infra.json']

In [ ]:
# 비교할 두 JSON 파일 경로를 지정합니다.
original_path = './iam_policy_devops_infra.json'
modified_path = './terraform/#policy/iam_policy_devops_infra.json'


In [15]:
def compare_sid_and_actions(original_path, modified_path):
    # JSON 파일을 읽어옵니다.
    with open(original_path, 'r', encoding='utf-8') as original_file, open(modified_path, 'r', encoding='utf-8') as modified_file:
        original_json = json.load(original_file)
        modified_json = json.load(modified_file)
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 추출
    original_statements = original_json.get("Statement", [])
    modified_statements = modified_json.get("Statement", [])
    
    original_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in original_statements}
    modified_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in modified_statements}
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 차이 비교
    added_sids = [sid for sid in modified_sid_actions.keys() if sid not in original_sid_actions]
    removed_sids = [sid for sid in original_sid_actions.keys() if sid not in modified_sid_actions]
    
    # 추가된 Sid와 해당 Action 값 반환
    added_sid_actions = {}
    for sid in added_sids:
        added_sid_actions[sid] = modified_sid_actions[sid]
    
    return added_sid_actions

def convert_pattern(pattern_string):
    # '*'를 '.'으로 대체하고 '^'와 '$'를 문자열의 시작과 끝으로 추가하여 변환
    converted_pattern = re.sub(r'\*', '.*', pattern_string)
    converted_pattern = f'^{converted_pattern}$'
    return converted_pattern

In [28]:
import os
import config
import json
import openpyxl

for file in original_files:
    if file in modify_files:
        original_path = os.path.join(config.iam_policy_data.original_path, file)
        modified_path = os.path.join(config.iam_policy_data.modified_path, file)

        added_sid_actions = compare_sid_and_actions(original_path, modified_path)

        

In [18]:
# 엑셀 파일 열기
workbook = openpyxl.load_workbook('./api_operations.xlsx')
worksheet = workbook['AWS_api_operations']


# 추가된 Sid와 해당 Action 값이 포함된지 확인하고 업데이트
for sid, action_list in added_sid_actions.items():    
    for action in action_list:
        try:
            # 엑셀에서 action을 찾는 로직을 구현해야 합니다.
            # 셀을 찾은 후 셀의 값을 업데이트하거나 새로운 행을 추가해야 합니다.
            pass

        except Exception as e:
            print(f"The {action} is wildcard type. Bring the {action} policy all list.")
            
            converted_pattern = convert_pattern(action)
            wildcard_policy_all = re.compile(converted_pattern)

            # 셀을 찾는 로직을 구현해야 합니다.
            # 정규 표현식과 일치하는 모든 셀을 찾아서 업데이트하거나 새로운 행을 추가해야 합니다.
            pass

# 엑셀 파일 저장
workbook.save('your_updated_excel_file.xlsx')

{'LambdaForDevOps': ['lambda:Get*',
  'lambda:List*',
  'lambda:CreateAlias',
  'lambda:DeleteFunction'],
 'RDSForDevOps': ['rds:*'],
 'TagForDevOps': ['tag:getResources',
  'tag:getTagKeys',
  'tag:getTagValues',
  'tag:TagResources',
  'tag:UntagResources'],
 'ComputerOptimizer': ['compute-optimizer:*'],
 'SNSandChatbotforDevOps': ['SNS:*', 'chatbot:*']}

In [35]:
# 엑셀 파일 열기
workbook = openpyxl.load_workbook('./api_operations.xlsx')
worksheet = workbook['AWS_api_operations']


In [ ]:
import re
import json
import time
import openpyxl

def convert_pattern(pattern_string):
    # '*'를 '.'으로 대체하고 '^'와 '$'를 문자열의 시작과 끝으로 추가하여 변환
    converted_pattern = re.sub(r'\*', '.*', pattern_string)
    converted_pattern = f'^{converted_pattern}$'
    return converted_pattern

# 비교할 두 JSON 파일 경로를 지정합니다.
original_path = './iam_policy_devops_infra.json'
modified_path = './terraform/#policy/iam_policy_devops_infra.json'

# JSON 파일의 추가된 Sid와 해당 Action 값 비교 함수를 호출합니다.
added_sid_actions = compare_sid_and_actions(original_path, modified_path)

# 엑셀 파일 열기
workbook = openpyxl.load_workbook('./api_operations.xlsx')
worksheet = workbook['AWS_api_operations']

# 추가된 Sid와 해당 Action 값이 포함된지 확인하고 업데이트
for sid, action_list in added_sid_actions.items():
    for action in action_list:
        try:
            # action을 포함하는 셀을 찾습니다.
            for row in worksheet.iter_rows():
                for cell in row:
                    if cell.value == action:
                        # 열 번호는 1부터 시작하므로 +1을 해줍니다.
                        worksheet.cell(row=cell.row, column=cell.column + 4, value=action)  # Action 값을 Sid 옆의 다음 열에 업데이트
                        print(f"{action} Update Complete")
                        # break
                else:
                    continue
                # break

        except Exception as e:
            print(f"The {action} is wildcard type. Bring the {action} policy all list.")

            converted_pattern = convert_pattern(action)
            wildcard_policy_all = re.compile(converted_pattern)

            # 정규 표현식과 일치하는 모든 셀을 찾습니다.
            for row in worksheet.iter_rows():
                for cell in row:
                    if cell.value is not None and wildcard_policy_all.match(cell.value):
                        if worksheet.cell(row=cell.row, column=cell.column + 6).value == '상':
                            worksheet.cell(row=cell.row, column=cell.column + 10, value='중')
                            print(f"{cell.value} Update Complete")
                            update_count += 1
                            if update_count >= 20:
                                print("20 updates completed. Waiting for 1 minute...")
                                for i in range(60):
                                    print(f"{i + 1} seconds passed.")
                                    time.sleep(1)
                                print("1 minute has passed.")
                                update_count = 0  # 업데이트 카운트 초기화
                        break
                else:
                    continue
                break

            # 해당 action을 포함하는 셀이 없는 경우 새로운 행을 추가합니다.
            else:
                new_row_data = ['', '', action, '', '', '', '중', 'test']
                worksheet.append(new_row_data)

# 변경된 엑셀 파일 저장
workbook.save('your_updated_excel_file.xlsx')


In [46]:
## 2024/02/19

import os
import re
import json

import preprocessiong_iam_policy
import config

def compare_sid_and_actions(original_path, modified_path):
    # JSON 파일을 읽어옵니다.
    with open(original_path, 'r', encoding='utf-8') as original_file, open(modified_path, 'r', encoding='utf-8') as modified_file:
        original_json = json.load(original_file)
        modified_json = json.load(modified_file)
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 추출
    original_statements = original_json.get("Statement", [])
    modified_statements = modified_json.get("Statement", [])
    
    original_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in original_statements}
    modified_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in modified_statements}
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 차이 비교
    added_sids = [sid for sid in modified_sid_actions.keys() if sid not in original_sid_actions]
    removed_sids = [sid for sid in original_sid_actions.keys() if sid not in modified_sid_actions]
    
    # 추가된 Sid와 해당 Action 값 반환
    added_sid_actions = {}
    for sid in added_sids:
        added_sid_actions[sid] = modified_sid_actions[sid]
    
    return added_sid_actions

def convert_pattern(pattern_string):
    # '*'를 '.'으로 대체하고 '^'와 '$'를 문자열의 시작과 끝으로 추가하여 변환
    converted_pattern = re.sub(r'\*', '.*', pattern_string)
    converted_pattern = f'^{converted_pattern}$'
    return converted_pattern

In [47]:
original_files=preprocessiong_iam_policy.policy_file_name_list(config.iam_policy_data.original_path)
modify_files=preprocessiong_iam_policy.policy_file_name_list(config.iam_policy_data.modified_path)

In [57]:
for file in original_files:
    if file in modify_files:
        original_path = os.path.join(config.iam_policy_data.original_path, file)
        modified_path = os.path.join(config.iam_policy_data.modified_path, file)

{}
{}
{}
{}
{}
{'LambdaForDevOps': ['lambda:Get*', 'lambda:List*', 'lambda:CreateAlias', 'lambda:DeleteFunction'], 'RDSForDevOps': ['rds:*'], 'TagForDevOps': ['tag:getResources', 'tag:getTagKeys', 'tag:getTagValues', 'tag:TagResources', 'tag:UntagResources'], 'ComputerOptimizer': ['compute-optimizer:*'], 'SNSandChatbotforDevOps': ['SNS:*', 'chatbot:*']}


In [56]:
added_sid_actions

{'LambdaForDevOps': ['lambda:Get*',
  'lambda:List*',
  'lambda:CreateAlias',
  'lambda:DeleteFunction'],
 'RDSForDevOps': ['rds:*'],
 'TagForDevOps': ['tag:getResources',
  'tag:getTagKeys',
  'tag:getTagValues',
  'tag:TagResources',
  'tag:UntagResources'],
 'ComputerOptimizer': ['compute-optimizer:*'],
 'SNSandChatbotforDevOps': ['SNS:*', 'chatbot:*']}

In [ ]:
# 추가된 Sid와 해당 Action 값이 포함된지 확인하고 업데이트
for sid, action_list in added_sid_actions.items():
    for action in action_list:
        try:
            cell = worksheet.find(action)
            worksheet.update_cell(cell.row, cell.col + 4, action)  # Action 값을 Sid 옆의 다음 열에 업데이트
            print(f"{action} Update Complete")
            update_count += 1

        except Exception as e:
            print(f"The {action} is wildcard type. Bring the {action} policy all list.")
            
            converted_pattern = convert_pattern(action)
            wildcard_policy_all = re.compile(converted_pattern)

            cell_list = worksheet.findall(wildcard_policy_all)

            if cell_list != []:
                for cell in cell_list:
                    worksheet.find(cell.value)
                    if worksheet.row_values(cell.row)[6] == '상':
                        worksheet.update_cell(cell.row, cell.col + 4, '중')
                    print(f"{cell.value} Update Complete")
                    update_count += 1
                    if update_count >= 20:
                        print("20 updates completed. Waiting for 1 minute...")
                        for i in range(60):
                            print(f"{i+1} seconds passed.")
                            time.sleep(1)
                        print("1 minute has passed.")
                        update_count = 0    # 업데이트 카운트 초기화
            else:
                new_row_data = ['', '', action, '', '', '', '중', 'test']
                worksheet.append_row(new_row_data)


In [5]:
import pandas as pd
data = pd.read_excel('./api_operations.xlsx', sheet_name="AWS_api_operations")

In [6]:
action

NameError: name 'action' is not defined

In [113]:
mask = data.isin(['중'])
cell = data[mask].stack().index.tolist()[0]
cell

(0, 'Service Effect')

In [90]:
try:
    # 특정 값(action)을 포함하는 행을 찾습니다.
    mask = data.isin(["chatbot:*"])
    cell = data[mask].stack().index.tolist()[0]
    
    # 찾은 행의 인덱스와 열 이름을 사용하여 업데이트합니다.
    row_index, col_name = cell
    data.at[row_index, "Service Effect"] = "중"
    
    print(f"{action} Update Complete")

except IndexError:
    print(f"{action} not found in the DataFrame")

chatbot:* not found in the DataFrame


In [115]:
### 내일은 여기서 부터 정리 ######################################
### 2024/02/20 #############################################

import preprocessiong_iam_policy

# 데이터프레임에서 특정 값(action)을 찾아 업데이트하는 함수 정의
def update_dataframe_with_action(df, action, effect):
    try:
        # 특정 값(action)을 포함하는 행을 찾습니다.
        mask = df.isin([action])
        cell = df[mask].stack().index.tolist()[0]
        
        # 찾은 행의 인덱스와 열 이름을 사용하여 업데이트합니다.
        row_index, col_name = cell
        df.at[row_index, "Service Effect"] = effect
        
        print(f"{action} Update Complete")
    
    except IndexError:
        print(f"{action} not found in the DataFrame")

        converted_pattern = preprocessiong_iam_policy.convert_pattern(action)
        wildcard_policy_all = re.compile(converted_pattern)

        cell_list = preprocessiong_iam_policy.find_pattern_in_dataframe(df, wildcard_policy_all)
        
        if cell_list != []:
            for cell in cell_list:
                mask = df.isin([cell])
                cell = df[mask].stack().index.tolist()[0]

                row_index, col_name = cell
                df.at[row_index, "Service Effect"] = effect
                print(f"{df['Actions'][row_index]} Update Complete")
        else:
            new_row_data = ['', '', action, '', '', '', effect, 'test']
            df = df.append(new_row_data, ignore_index=True)

    preprocessiong_iam_policy.dataframe_to_excel(df, config.iam_policy_data.output_file_path)

                

In [ ]:
            if cell_list != []:
                for cell in cell_list:
                    worksheet.find(cell.value)
                    if worksheet.row_values(cell.row)[6] == '상':
                        worksheet.update_cell(cell.row, cell.col + 4, '중')
                    print(f"{cell.value} Update Complete")
                    update_count += 1
                    if update_count >= 20:
                        print("20 updates completed. Waiting for 1 minute...")
                        for i in range(60):
                            print(f"{i+1} seconds passed.")
                            time.sleep(1)
                        print("1 minute has passed.")
                        update_count = 0    # 업데이트 카운트 초기화
            else:
                new_row_data = ['', '', action, '', '', '', '중', 'test']
                worksheet.append_row(new_row_data)

In [7]:
action='rds:*'

In [8]:
import preprocessiong_iam_policy
import re

In [9]:
converted_pattern = preprocessiong_iam_policy.convert_pattern(action)

In [57]:
new_row_data = ['', '', action, '', '', '', '중', 'test']
data.loc[len(data)] = new_row_data

In [11]:
wildcard_policy_all = re.compile(converted_pattern)

In [12]:
matching_rows = data.apply(lambda row: row.astype(str).str.contains(wildcard_policy_all).any(), axis=1)

In [16]:
wildcard_policy_all

re.compile(r'^rds:.*$', re.UNICODE)

In [35]:
data["Actions"][]

0                           cloudwatch:*
1                               health:*
2                              support:*
3               s3:AbortMultipartUpload 
4              ec2:AcceptAddressTransfer
                      ...               
1426        iam:UploadSigningCertificate
1427              iam:UploadSSHPublicKey
1428                 ecr:UploadLayerPart
1429    ecr:ValidatePullThroughCacheRule
1430               ec2:WithdrawByoipCidr
Name: Actions, Length: 1431, dtype: object

In [37]:
for i, row in enumerate(matching_rows):
    if row == True:
        print(i, row, data["Actions"][i])
        

11 True rds:AddRoleToDBCluster
12 True rds:AddRoleToDBInstance
13 True rds:AddSourceIdentifierToSubscription
14 True rds:AddTagsToResource
25 True rds:ApplyPendingMaintenanceAction
52 True rds:AuthorizeDBSecurityGroupIngress
53 True rds:BacktrackDBCluster
69 True rds:CancelExportTask
81 True rds:CopyDBClusterParameterGroup
82 True rds:CopyDBClusterSnapshot
83 True rds:CopyDBParameterGroup
84 True rds:CopyDBSnapshot
85 True rds:CopyOptionGroup
144 True rds:CreateBlueGreenDeployment
145 True rds:CreateCustomDBEngineVersion
146 True rds:CreateDBCluster
147 True rds:CreateDBClusterEndpoint
148 True rds:CreateDBClusterParameterGroup
149 True rds:CreateDBClusterSnapshot
150 True rds:CreateDBInstance
151 True rds:CreateDBInstanceReadReplica
152 True rds:CreateDBParameterGroup
153 True rds:CreateDBProxy
154 True rds:CreateDBProxyEndpoint
155 True rds:CreateDBSecurityGroup
156 True rds:CreateDBSnapshot
157 True rds:CreateDBSubnetGroup
158 True rds:CreateEventSubscription
159 True rds:CreateGlob

In [38]:
import preprocessiong_iam_policy
matching_rows = data.apply(lambda row: row.astype(str).str.contains(wildcard_policy_all).any(), axis=1)

pattern_data = []

for i, row in enumerate(matching_rows):
    if row == True:
        pattern_data.append(data["Actions"][i])

In [39]:
pattern_data

['rds:AddRoleToDBCluster',
 'rds:AddRoleToDBInstance',
 'rds:AddSourceIdentifierToSubscription',
 'rds:AddTagsToResource',
 'rds:ApplyPendingMaintenanceAction',
 'rds:AuthorizeDBSecurityGroupIngress',
 'rds:BacktrackDBCluster',
 'rds:CancelExportTask',
 'rds:CopyDBClusterParameterGroup',
 'rds:CopyDBClusterSnapshot',
 'rds:CopyDBParameterGroup',
 'rds:CopyDBSnapshot',
 'rds:CopyOptionGroup',
 'rds:CreateBlueGreenDeployment',
 'rds:CreateCustomDBEngineVersion',
 'rds:CreateDBCluster',
 'rds:CreateDBClusterEndpoint',
 'rds:CreateDBClusterParameterGroup',
 'rds:CreateDBClusterSnapshot',
 'rds:CreateDBInstance',
 'rds:CreateDBInstanceReadReplica',
 'rds:CreateDBParameterGroup',
 'rds:CreateDBProxy',
 'rds:CreateDBProxyEndpoint',
 'rds:CreateDBSecurityGroup',
 'rds:CreateDBSnapshot',
 'rds:CreateDBSubnetGroup',
 'rds:CreateEventSubscription',
 'rds:CreateGlobalCluster',
 'rds:CreateIntegration',
 'rds:CreateOptionGroup',
 'rds:CreateTenantDatabase',
 'rds:DeleteBlueGreenDeployment',
 'rds:D

In [ ]:
import re
import json
import time

def compare_sid_and_actions(original_path, modified_path):
    # JSON 파일을 읽어옵니다.
    with open(original_path, 'r', encoding='utf-8') as original_file, open(modified_path, 'r', encoding='utf-8') as modified_file:
        original_json = json.load(original_file)
        modified_json = json.load(modified_file)
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 추출
    original_statements = original_json.get("Statement", [])
    modified_statements = modified_json.get("Statement", [])
    
    original_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in original_statements}
    modified_sid_actions = {stmt.get("Sid"): stmt.get("Action") for stmt in modified_statements}
    
    # 원본 파일과 수정 파일에서 "Statement" 리스트의 "Sid" 차이 비교
    added_sids = [sid for sid in modified_sid_actions.keys() if sid not in original_sid_actions]
    removed_sids = [sid for sid in original_sid_actions.keys() if sid not in modified_sid_actions]
    
    # 추가된 Sid와 해당 Action 값 반환
    added_sid_actions = {}
    for sid in added_sids:
        added_sid_actions[sid] = modified_sid_actions[sid]
    
    return added_sid_actions

def convert_pattern(pattern_string):
    # '*'를 '.'으로 대체하고 '^'와 '$'를 문자열의 시작과 끝으로 추가하여 변환
    converted_pattern = re.sub(r'\*', '.*', pattern_string)
    converted_pattern = f'^{converted_pattern}$'
    return converted_pattern

# 비교할 두 JSON 파일 경로를 지정합니다.
original_path = './iam_policy_devops_infra.json'
modified_path = './terraform/#policy/iam_policy_devops_infra.json'

# JSON 파일의 추가된 Sid와 해당 Action 값 비교 함수를 호출합니다.
# compare_origin_and_modify(original_path, modified_path)
added_sid_actions = compare_sid_and_actions(original_path, modified_path)

# 구글 시트에 연결
scope = scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_keyfile = './python-iam-policy-update-d8f00ffa6f0b.json'  # JSON 키 파일의 경로
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile, scope)
gc = gspread.authorize(credentials)

# 구글 시트 문서 및 워크시트 열기
sheet_url = 'https://docs.google.com/spreadsheets/d/1N6EiipYeeQdEKR24mvaAFm3jAJF3d8yDNPVc47ZqgRU/edit?usp=sharing'
doc = gc.open_by_url(sheet_url)
worksheet = doc.worksheet('AWS_api_operations')

update_count = 0

# 추가된 Sid와 해당 Action 값이 포함된지 확인하고 업데이트
for sid, action_list in added_sid_actions.items():    
    for action in action_list:
        try:
            cell = worksheet.find(action)
            worksheet.update_cell(cell.row, cell.col + 4, action)  # Action 값을 Sid 옆의 다음 열에 업데이트
            print(f"{action} Update Complete")
            update_count += 1

        except Exception as e:
            print(f"The {action} is wildcard type. Bring the {action} policy all list.")
            
            converted_pattern = convert_pattern(action)
            wildcard_policy_all = re.compile(converted_pattern)

            cell_list = worksheet.findall(wildcard_policy_all)

            if cell_list != []:
                for cell in cell_list:
                    worksheet.find(cell.value)
                    if worksheet.row_values(cell.row)[6] == '상':
                        worksheet.update_cell(cell.row, cell.col + 4, '중')
                    print(f"{cell.value} Update Complete")
                    update_count += 1
                    if update_count >= 20:
                        print("20 updates completed. Waiting for 1 minute...")
                        for i in range(60):
                            print(f"{i+1} seconds passed.")
                            time.sleep(1)
                        print("1 minute has passed.")
                        update_count = 0    # 업데이트 카운트 초기화
            else:
                new_row_data = ['', '', action, '', '', '', '중', 'test']
                worksheet.append_row(new_row_data)
